<a href="https://colab.research.google.com/github/giuliovv/ANNDL_competition_1/blob/master/giulio_models/xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os

colab = "True" #@param ['True','False']
if colab == "True":
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/Colab Notebooks
  if not os.path.isdir('training'):
    !unzip dataset.zip

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks


In [30]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [31]:
dataset_dir = 'training'

Let the game begin

In [32]:
import tensorflow as tf

In [33]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [36]:
y = tf.keras.utils.to_categorical(range(len(labels)))
y

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

In [37]:
import shutil

get_plants = False

if not "test" in os.listdir():
  print("No test directory!")
  for label in labels:
    print(label)
    image_names = [pic for pic in os.listdir("training/"+label)]
    train_images, test_images = train_test_split(image_names, test_size=0.05)
    if not 'test/'+label in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test/'+label+'/')
      except OSError:
        print("Failed to create a dir for ", '/test/'+label+'/')
      else:
        print("Success creating folder ", label)
    for name in test_images:
      shutil.move(os.path.abspath(os.getcwd())+'/training/'+label+'/'+name, os.path.abspath(os.getcwd())+'/test/'+label+'/'+name)
  print("Transfered all testing data!")

if get_plants:
  ! pip install -q tfds-nightly
  import tensorflow_datasets as tfds
  from PIL import Image
  ds = tfds.load('plant_village', split='train')
  builder = tfds.builder('plant_village')
  info = builder.info
  labels_tf = info.features["label"].names
  if not 'test_expanded/' in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/')
      except OSError:
        print("Failed to create a dir for /test_expanded/")
      else:
        print("Success creating folder test_expanded")
  image_number = 54303
  ds = ds.take(54303)
  for el in ds:
    label_new_ds = labels_tf[el["label"].numpy()]
    for just_fruit_name in labels:
      if just_fruit_name in label_new_ds:
        if not just_fruit_name in os.listdir("test_expanded"):
          try:
            os.makedirs(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+'/')
          except OSError:
            print("Failed to create a dir for ", '/test_expanded/'+just_fruit_name+'/')
          else:
            print("Success creating folder ", just_fruit_name)
        im = Image.fromarray(el["image"].numpy())
        im.save(os.path.abspath(os.getcwd())+'/test_expanded/'+just_fruit_name+"/"+str(image_number)+".jpeg")
        break
    if image_number % 1000 == 0:
      print(image_number)
    image_number += 1


In [38]:
import random

def preproc(img):
  '''Add random noise to an image'''
  VARIABILITY = 50
  deviation = VARIABILITY*random.random()
  noise = np.random.normal(0, deviation, img.shape)
  img += noise
  np.clip(img, 0., 255.)
  X = tf.keras.applications.inception_v3.preprocess_input(img)
  return X

In [39]:
BATCH_SIZE = 64
VALIDATE_BATCH_SIZE = BATCH_SIZE

train_generator = ImageDataGenerator(
## when rotating some pixels remain unfilled, to fill them the default function is to take the "nearest pixel color", this maybe screws up the learning process so I left it for later training
                                    rotation_range=90,
                                    fill_mode='nearest',
                                    brightness_range=(0.2,1.8),
                                    channel_shift_range=150,
                                    shear_range=0.5,
                                    zoom_range=0.5,
                                    width_shift_range=0.1, 
                                    height_shift_range=0.1,
                                    horizontal_flip=True, 
                                    vertical_flip=True,
                                    validation_split=0.2,
                                    preprocessing_function=tf.keras.applications.xception.preprocess_input) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)

traingen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='training',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

validgen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='validation',
                                              batch_size=VALIDATE_BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

testgen = test_generator.flow_from_directory('test',
                                            target_size=(256, 256),
                                            class_mode='categorical',
                                            classes=labels,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=42)

Found 13238 images belonging to 14 classes.
Found 3301 images belonging to 14 classes.
Found 877 images belonging to 14 classes.


In [40]:
# create the base pre-trained model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256,3))

In [41]:
n_classes = len(labels)

In [42]:
load = True

if load:
  model = tf.keras.models.load_model("xception_only_top")

In [43]:
from keras.layers import LeakyReLU
lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01)

if not load:
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation=lrelu, name="first")(x)
  x = Dense(512, activation=lrelu, name="second")(x)
  x = Dense(512, activation=lrelu, name="third")(x)
  # and a logistic layer -- let's say we have n_classes classes
  predictions = Dense(n_classes, activation='softmax', name="last")(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)


In [44]:
if not load:

  # first: train only the top layers (which were randomly initialized)
  # i.e. freeze all convolutional InceptionV3 layers
  for layer in base_model.layers:
      layer.trainable = False

  optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

  # compile the model (should be done *after* setting layers to non-trainable)
  model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)
    callbacks.append(es_callback)
    
#LRPlateau
lr_plateau_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=2,
    min_lr=0,
)  

callbacks.append(lr_plateau_callback)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

callbacks.append(tensorboard_callback)

backup = tf.keras.callbacks.experimental.BackupAndRestore(
    "xception_top_only_backup"
)

In [46]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [47]:
%tensorboard --logdir "./logs"

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 1340.

In [48]:
n_steps = traingen.samples / BATCH_SIZE
n_val_steps = validgen.samples / VALIDATE_BATCH_SIZE

if not load:
  # train the model on the new data for a few epochs
  # 18 epochs with dropout
  model.fit(traingen, epochs=12, steps_per_epoch=n_steps, batch_size=BATCH_SIZE, validation_data=validgen, callbacks=callbacks+[backup])

In [49]:
if not load:
  model.save("xception_only_top")

In [50]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_2
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_4
13 block2_pool
14 batch_normalization_4
15 add_12
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_5
23 block3_pool
24 batch_normalization_5
25 add_13
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_6
33 block4_pool
34 batch_normalization_6
35 add_14
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_15
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_se

In [51]:
load_second = False
if load_second:
  model = tf.keras.models.load_model("xception")

In [52]:
if not load_second:
  # we chose to train the top 2 inception blocks, i.e. we will freeze
  # the first 249 layers and unfreeze the rest:
  for layer in model.layers[:50]:
    layer.trainable = False
  for layer in model.layers[50:]:
    layer.trainable = True

  # we need to recompile the model for these modifications to take effect
  # we use SGD with a low learning rate
  from tensorflow.keras.optimizers import Adam
  model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

backup2 = tf.keras.callbacks.experimental.BackupAndRestore(
    "xception_second_part_backup"
)

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(traingen, epochs=15, steps_per_epoch=n_steps,  batch_size=BATCH_SIZE, validation_data=validgen, callbacks=callbacks+[backup2])

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/15
206/206 [==============================] - 496s 2s/step - loss: 0.4273 - accuracy: 0.8667 - val_loss: 0.4123 - val_accuracy: 0.8773 - lr: 1.0000e-04
Epoch 2/15
206/206 [==============================] - 481s 2s/step - loss: 0.1971 - accuracy: 0.9355 - val_loss: 0.2413 - val_accuracy: 0.9261 - lr: 1.0000e-04
Epoch 3/15
206/206 [==============================] - 478s 2s/step - loss: 0.1430 - accuracy: 0.9552 - val_loss: 0.2471 - val_accuracy: 0.9221 - lr: 1.0000e-04
Epoch 4/15
206/206 [==============================] - 477s 2s/step - loss: 0.1228 - accuracy: 0.9605 - val_loss: 0.1880 - val_accuracy: 0.9446 - lr: 1.0000e-04
Epoch 5/15
206/206 [==============================] - 478s 2s/step - loss: 0.0988 - accuracy: 0.9680 - val_loss: 0.2323 - val_accuracy: 0.9315 - lr: 1.0000e-04
Epoch 6/15
206/206 [==============================] - 478s 2s/step - loss: 0.1005 - accuracy: 0.9671 - val_loss: 0.1774 - val_accuracy: 0.9452 - lr: 1.0000e-04
Epoch 7/15
206/206 [====================

In [53]:
model.save("xception")

INFO:tensorflow:Assets written to: xception/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [54]:
model.evaluate(testgen)

877/877 [==============================] - 287s 327ms/step - loss: 0.0057 - accuracy: 0.9989


[0.005700704175978899, 0.9988597631454468]